In [8]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.selenium_manager import SeleniumManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from googletrans import Translator
from google_trans_new import google_translator  
from deep_translator import GoogleTranslator
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pandas as pd
from sqlalchemy import create_engine
import warnings
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, StringType
import pyspark.pandas as ps
warnings.filterwarnings('ignore')

In [2]:
def translator(s,source='auto'):
        return GoogleTranslator(source=source, target='en').translate(s) 

In [3]:
input_params_dropdown_by_id = {}
input_params_dropdown_by_id['dbselect'] = '2023' #Select Year
input_params_dropdown_by_id['district_id'] = 'मुंबई उपनगर' #District
input_params_dropdown_by_id['taluka_id'] = 'अंधेरी' #Taluka
input_params_dropdown_by_id['village_id'] = 'बांद्रा' #Village

input_params_text_by_id = {}
input_params_text_by_id['free_text'] = '2023' 

url = 'https://pay2igr.igrmaharashtra.gov.in/eDisplay/Propertydetails/index' 
driver = webdriver.Chrome() 
driver.implicitly_wait(10)
driver.get(url)

for key, value in input_params_dropdown_by_id.items():
	print(key,value)
	dropdown = driver.find_element(by=By.ID,value=key)
	dropdown_select = Select(dropdown)
	for option in dropdown_select.options:
		if option.text == value:
			option.click()
			print(f'Clicked {option.text} for {key}')
			break
	driver.implicitly_wait(2)

#Input reg year
driver.find_element(by=By.ID, value='free_text').send_keys(input_params_text_by_id['free_text'])
driver.implicitly_wait(2)

#Input Captcha
captcha = input()
driver.find_element(by=By.ID, value='cpatchaTextBox').send_keys(captcha)
print(f'Entered Captcha {captcha}')
driver.implicitly_wait(2)

#Submit to get result set
driver.find_element(by=By.ID,value='submit').click()
driver.implicitly_wait(10)


#Click on 50 Pages
dropdown = driver.find_element(by=By.NAME, value='tableparty_length')
dropdown_select = Select(dropdown)
for option in dropdown_select.options:
	if option.text == 'All':
		option.click()
		print('Set to All Records')
		break

print('So Far, So Good')

dbselect 2023
Clicked 2023 for dbselect
district_id मुंबई उपनगर
Clicked मुंबई उपनगर for district_id
taluka_id अंधेरी
Clicked अंधेरी for taluka_id
village_id बांद्रा
Clicked बांद्रा for village_id
Entered Captcha TMSHRU
Set to All Records
So Far, So Good


In [4]:
records_raw = pd.DataFrame(columns=['अनु क्र.','दस्त क्र.','दस्त प्रकार','दू. नि. कार्यालय','वर्ष','लिहून देणार','लिहून घेणार','इतर माहीती','सूची क्र. २'])

for index, table in enumerate(driver.find_elements(by=By.ID, value='tbdata')):
    data = [item.text if item.text != 'सूची क्र. २' else item.find_element(by=By.TAG_NAME,value='a').get_attribute('href') for item in table.find_elements(by=By.XPATH, value=".//*[self::td or self::th]")]
    records_raw.loc[len(records_raw)] = data
    # print(data)
    print(f'{index} Row appended')

records_raw
records_translate = records_raw.copy()

0 Row appended
1 Row appended
2 Row appended
3 Row appended
4 Row appended
5 Row appended
6 Row appended
7 Row appended
8 Row appended
9 Row appended
10 Row appended
11 Row appended
12 Row appended
13 Row appended
14 Row appended
15 Row appended
16 Row appended
17 Row appended
18 Row appended
19 Row appended
20 Row appended
21 Row appended
22 Row appended
23 Row appended
24 Row appended
25 Row appended
26 Row appended
27 Row appended
28 Row appended
29 Row appended
30 Row appended
31 Row appended
32 Row appended
33 Row appended
34 Row appended
35 Row appended
36 Row appended
37 Row appended
38 Row appended
39 Row appended
40 Row appended
41 Row appended
42 Row appended
43 Row appended
44 Row appended
45 Row appended
46 Row appended
47 Row appended
48 Row appended
49 Row appended
50 Row appended
51 Row appended
52 Row appended
53 Row appended
54 Row appended
55 Row appended
56 Row appended
57 Row appended
58 Row appended
59 Row appended
60 Row appended
61 Row appended
62 Row appended
63

In [6]:
# PostgreSQL database connection parameters
db_user = 'postgres'
db_password = 'Sunrise12345'
db_host = '127.0.0.1'
db_port = '5432'
db_name = 'propReturns'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
table_name = 'record_details_raw'

records_raw.to_sql(table_name, engine, if_exists='replace', index=True)

print(f"{len(records_raw)} DataFrame has been inserted into the '{table_name}' table.")

1209 DataFrame has been inserted into the 'record_details_raw' table.


In [8]:
#Approach Via Pandas
records_translate = records_raw.copy()
records_translate.columns = [translator(col,source='auto') for col in records_translate.columns]
cols_to_translate = [
    'diarrhea type',
    'Du. Prohibit. Office',
    'Will write', 
    'Will write down', 
    'Other information'
    ]

records_translated = pd.DataFrame(columns=records_translate.columns)
batch_size = 100
start = 0
for batch in range(start,len(records_translate),batch_size):
    print(f'Batch -> [{batch},{batch+batch_size}]')
    temp_df = records_translate.iloc[batch:batch+batch_size,:].copy()
    for col in cols_to_translate:
        print(f'Translating {col}')
        temp_df[col] = temp_df[col].apply(lambda x: translator(x,source='hi'))
        print(f'Done')
    records_translated = pd.concat([records_translated,temp_df])
records_translated.rename(columns={'Will write':'buyer_name','Will write down':'seller_name'},inplace=True)

Batch -> [0,100]
Translating diarrhea type
Done
Translating Du. Prohibit. Office
Done
Translating Will write


RequestError: Request exception can happen due to an api connection error. Please check your connection and try again

In [9]:
# Approach Via Spark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark = SparkSession.builder.getOrCreate()
spark

In [10]:
# del spark_df 
spark_df = spark.createDataFrame(records_raw.copy())

In [11]:
new_names = []
for col in records_raw.columns:
    new_names.append(translator(col,source='auto'))

new_names = [x.replace('.','') for x in new_names]
spark_df = spark_df.toDF(*new_names)

In [12]:
spark_df.show()

+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|Sl no|Diarrhea no|       diarrhea type|Du Prohibit Office|      Year|          Will write|     Will write down|   Other information|           List no 2|
+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    1|      13217|           भाडेपट्टा|सह दु.नि. अंधेरी 7|25/07/2023|1) श्रीमती चंद्रक...|1) महाराष्ट़ हाऊस...|1) इतर माहिती: पि...|https://pay2igr.i...|
|    2|       4286|       विकसनकरारनामा|सह दु.नि. अंधेरी 7|10/03/2023|1) अजित सिंह करता...|1) नाविश रियल्टी ...|1) इतर माहिती: जम...|https://pay2igr.i...|
|    3|       2449|             सेल डीड|सह दु.नि. अंधेरी 7|09/02/2023|1) नूपुर अनिल कळक...|1) हर्ष सतपाल मल्...|1) इतर माहिती: सद...|https://pay2igr.i...|
|    4|       8691|66-नोटीस ऑफ़ लिस प...|सह दु.नि. अंधेरी 7|22/05/2023|

In [13]:
spark_df.printSchema()

root
 |-- Sl no: string (nullable = true)
 |-- Diarrhea no: string (nullable = true)
 |-- diarrhea type: string (nullable = true)
 |-- Du Prohibit Office: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Will write: string (nullable = true)
 |-- Will write down: string (nullable = true)
 |-- Other information: string (nullable = true)
 |-- List no 2: string (nullable = true)



In [24]:
from googletrans import Translator
def translator_v2(s,source='auto'):
        translator = Translator()
        return translator.translate(s, src='hi',dest='en').text 

translate_udf = udf(translator_v2,StringType())

In [54]:
translate_udf_og = udf(translator,StringType())

In [55]:
spark_df.withColumn('Will write down', translate_udf_og(spark_df['Will write down'])).show()

+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|Sl no|Diarrhea no|       diarrhea type|Du Prohibit Office|      Year|          Will write|     Will write down|   Other information|           List no 2|
+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    1|      13217|           भाडेपट्टा|सह दु.नि. अंधेरी 7|25/07/2023|1) श्रीमती चंद्रक...|1) Hanmant Dhanwa...|1) इतर माहिती: पि...|https://pay2igr.i...|
|    2|       4286|       विकसनकरारनामा|सह दु.नि. अंधेरी 7|10/03/2023|1) अजित सिंह करता...|1) Paresh Ranchho...|1) इतर माहिती: जम...|https://pay2igr.i...|
|    3|       2449|             सेल डीड|सह दु.नि. अंधेरी 7|09/02/2023|1) नूपुर अनिल कळक...|1) Harsh Satpal M...|1) इतर माहिती: सद...|https://pay2igr.i...|
|    4|       8691|66-नोटीस ऑफ़ लिस प...|सह दु.नि. अंधेरी 7|22/05/2023|

In [56]:
cols_to_translate = [
    'diarrhea type',
    'Du Prohibit Office',
    'Will write', 
    # 'Will write down', 
    'Other information'
    ]

for column in cols_to_translate:
    print(column)
    # df_transformed = spark_df.withColumn(column, translate_udf(col(column)))
    #spark_df.withColumn(column, translate_udf(spark_df[column])).show()
    spark_df = spark_df.withColumn(column, translate_udf(spark_df[column]))

spark_df = spark_df.withColumn('Will write down', translate_udf_og(spark_df['Will write down']))


diarrhea type
Du Prohibit Office
Will write
Other information


In [57]:
spark_df.show()

+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|Sl no|Diarrhea no|       diarrhea type|Du Prohibit Office|      Year|          Will write|     Will write down|   Other information|           List no 2|
+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    1|      13217|               lease|Co. D.N. Andheri 7|25/07/2023|1) Bhavin Sheth, ...|1) Hanmant Dhanwa...|1) Other informat...|https://pay2igr.i...|
|    2|       4286|development agree...|Co. D.N. Andheri 7|10/03/2023|1) Ajit Singh Kar...|1) Paresh Ranchho...|1) Other informat...|https://pay2igr.i...|
|    3|       2449|           sale deed|Co. D.N. Andheri 7|09/02/2023|1) Nupur Anil Kal...|1) Harsh Satpal M...|1) Other Informat...|https://pay2igr.i...|
|    4|       8691|66-Notice of list...|Co. D.N. Andheri 7|22/05/2023|

In [59]:
spark_df.printSchema()

root
 |-- Sl no: string (nullable = true)
 |-- Diarrhea no: string (nullable = true)
 |-- diarrhea type: string (nullable = true)
 |-- Du Prohibit Office: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Will write: string (nullable = true)
 |-- Will write down: string (nullable = true)
 |-- Other information: string (nullable = true)
 |-- List no 2: string (nullable = true)



In [61]:
spark_df = spark_df.withColumnRenamed(existing='Will write',new='buyer_name')
spark_df = spark_df.withColumnRenamed(existing='Will write down',new='seller_name')

In [70]:
spark_df.printSchema()

root
 |-- Sl no: string (nullable = true)
 |-- Diarrhea no: string (nullable = true)
 |-- diarrhea type: string (nullable = true)
 |-- Du Prohibit Office: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- buyer_name: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- Other information: string (nullable = true)
 |-- List no 2: string (nullable = true)



In [72]:
new_df = spark_df

In [91]:
spark_df.write.format("csv").mode('overwrite').save("/tmp/spark_output/zipcodes")

Py4JJavaError: An error occurred while calling o831.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\winutils\bin -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache

In [89]:
# df_1 = new_df.select([
#                       'Sl no', 
#                       'Diarrhea no'
#                       ]).toPandas()

print('1')
# df_2 = new_df.select([
#                       'Sl no', 
#                       'diarrhea type' 
#                       ]).toPandas()

print('2')
# df_3 = new_df.select([
#                       'Sl no', 
#                       'Du Prohibit Office'
#                      ]).toPandas()

print('3')
# df_4 = new_df.select([
#                       'Sl no',
#                       'Year'
#                       ]).toPandas()

print('4')
df_5 = spark_df.select([
                      'Sl no',
                      'buyer_name',
                      ]).toPandas()

print('5')
df_6 = new_df.select([
                      'Sl no',
                      'seller_name',
                      ]).toPandas()

print('6')
df_7 = new_df.select([
                      'Sl no',
                      'Other information',
                      ]).toPandas()
print('7')
df_8 = new_df.select([
                      'Sl no',
                      'List no 2',
                      ]).toPandas()

1
2
3
4


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\ayush\AppData\Local\Temp\ipykernel_23996\1038912717.py", line 4, in translator_v2
  File "c:\Users\ayush\anaconda3\lib\site-packages\googletrans\client.py", line 213, in translate
    translated = ''.join([d[0] if d[0] else '' for d in data[0]])
TypeError: 'NoneType' object is not iterable


In [82]:
check_df = new_df.select(['Sl no', 'Diarrhea no', 'diarrhea type', 'seller_name', 'Du Prohibit Office','Year','buyer_name','seller_name','Other information',
                          'List no 2']).toPandas()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\ayush\AppData\Local\Temp\ipykernel_23996\1038912717.py", line 4, in translator_v2
  File "c:\Users\ayush\anaconda3\lib\site-packages\googletrans\client.py", line 213, in translate
    translated = ''.join([d[0] if d[0] else '' for d in data[0]])
TypeError: 'NoneType' object is not iterable


In [ ]:
check_df

,seller_name
0,"1) Hanmant Dhanware, Executive Engineer, Mumba..."
1,"1) Paresh Ranchhod Patel, partner of Navish Re..."
2,1) Harsh Satpal Malhotra
3,
4,1) Shama Subhash Oberoi
...,...
1204,1) Muhammad Kunhi KC
1205,1) Rashmi Satish Kevalramani\n2) Satish Kevalr...
1206,1) Jeram Chamadia (HUF) Laxman
1207,1) Karim B. pasture


In [62]:
# Save the DataFrame to a table in PostgreSQL
table_name = "record_details_translated"  # Replace with your desired table name
jdbc_url = "jdbc:postgresql://127.0.0.1:5432/propReturns"  # Replace with your PostgreSQL connection details
properties = {
    "user": "postgres",    # Replace with your PostgreSQL username
    "password": "Sunrise12345",  # Replace with your PostgreSQL password
    "driver": "org.postgresql.Driver"
}

spark_df.write \
    .mode("overwrite") \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .option("user", properties["user"]) \
    .option("password", properties["password"]) \
    .option("driver", properties["driver"]) \
    .save()

Py4JJavaError: An error occurred while calling o444.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:593)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:526)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:2

In [ ]:
table_name = 'record_details_translated'
records_translate.to_sql(table_name, engine, if_exists='replace', index=True)
print(f"{len(records_translate)} DataFrame has been inserted into the '{table_name}' table.")

50 DataFrame has been inserted into the 'record_details_translated' table.
